In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
import json

In [2]:
cd ../data/

/srv/home/christinedk/wp_internship/data


In [11]:
ls

autobiography.json  data_export_1402/
blp/                namespaces.csv
blp_articles.csv    revision_history/
categories.csv      revisions_with_templates_relaibility_with_stats.csv.gz


In [4]:
!gunzip tmp-autobiography-meta-info.csv.gz

gzip: tmp-autobiography-meta-info.csv.gz: No such file or directory


In [14]:
ls -lh /srv/home/christinedk/wp_internship/data/revision_history/

total 1.7G
-rw-r--r-- 1 christinedk wikidev 603M Feb 19 19:43 tmp-advert-meta-info.csv.gz
-rw-r--r-- 1 christinedk wikidev 136M Feb 19 19:40 tmp-autobiography-meta-info.csv
-rw-r--r-- 1 christinedk wikidev  54M Feb 19 19:43 tmp-fanpov-meta-info.csv.gz
-rw-r--r-- 1 christinedk wikidev 289M Feb 19 19:39 tmp-peacock-meta-info.csv.gz
-rw-r--r-- 1 christinedk wikidev 603M Feb 19 19:37 tmp-weasel-meta-info.csv.gz


In [15]:
!gunzip *.gz

In [5]:
with open("/srv/home/christinedk/wp_internship/data/revision_history/tmp-fanpov-meta-info.csv",'r') as my_file:
    text = [k.split('\t') for k in my_file.readlines()]
    
text = [i[:16] for i in text if len(i) >= 16]

KeyboardInterrupt: 

In [19]:
blp = pd.read_csv('/srv/home/christinedk/wp_internship/data/namespaces.csv')
blp = blp.query('page_namespace in (0,1)')

In [20]:
text_blp = [i for i in tqdm(text[1:]) if int(i[2]) in blp.page_id.values]

100%|██████████| 855402/855402 [26:30<00:00, 537.83it/s]


In [21]:
revisions = pd.DataFrame(text_blp, columns=text[0])
revisions['page_namespace'] = revisions['page_namespace'].astype(int)
revisions = revisions.query('page_namespace in (0,1)')

In [30]:
def get_edits_pre_tag(data):
    ix = data['has_template'].idxmax()

    return data.loc[:ix]

In [31]:
revisions = revisions.sort_values(by=['page_id','revision_id'], ascending=True)
revisions['has_template'] = revisions['has_template'].fillna(0).astype(int)
pre_edit = revisions.groupby('page_id').apply(get_edits_pre_tag)
pre_edit.reset_index(drop=True, inplace=True)

In [32]:
revisions.groupby(['page_namespace'])['page_id'].count()

page_namespace
0    631111
1     67967
Name: page_id, dtype: int64

In [33]:
revisions.groupby(['page_namespace'])['page_id'].nunique()

page_namespace
0    3643
1    3643
Name: page_id, dtype: int64

In [34]:
# persist

In [35]:
revisions.to_csv('/srv/home/christinedk/wp_internship/data/autobiography.csv',index=False)

In [36]:
with open('//srv/home/christinedk/wp_internship/data/autobiography.json','w') as f:
    json.dump(revisions.to_dict(orient='records'),f)